# Sort on institutions

In [175]:
# libraries
import pandas as pd
import numpy as np
import re
from itertools import chain
pd.options.mode.chained_assignment = None
import folium
import shapely.geometry as sg

# data load
df = pd.read_csv('data/kkorg_enheder_offentlig.csv', delimiter=';')

In [176]:
# Apply lambda function to each row to extract longitude and latitude
def extract_coordinates(row):
    # Define a regular expression pattern to extract coordinates
    pattern = r'[\d.]+'

    matches = re.findall(pattern, row)
    if len(matches) >= 2:
        return pd.Series((float(matches[0]), float(matches[1])))
    else:
        return pd.Series((None, None))


# Save the extracted coordinates in two new columns
df['wkb_geometry'] = df['wkb_geometry'].astype(str)
df[['longitude', 'latitude']] = df['wkb_geometry'].apply(extract_coordinates)

In [177]:
# find the rows where the column 'enhedstype' is 'Vuggestuer', 'Dagplejemødre', 'Børnehaver', 'Folkeskoler o.lign.', 'Aldersintegrerede institutioner'
filter_df = df[df['enhedstype'].isin(['Vuggestuer', 'Dagplejemødre', 'Børnehaver', 'Folkeskoler o.lign.', 'Aldersintegrerede institutioner'])]
filter_df['type_split'] = filter_df['type'].str.split(', ')
filter_df = filter_df.explode('type_split')
filter_df = filter_df[filter_df['type_split'].str.contains(r'Vuggestue|\bDagpleje\b|Flerbørnsdagpleje|Børnehave|Distriktsskole')]
filter_df = filter_df.reset_index(drop=True)

# make english names for the types
filter_df['type_EN'] = filter_df['type_split'].replace({'Vuggestue': 'Nursery', 'Dagpleje': 'Daycare', 'Flerbørnsdagpleje': 'Daycare', 'Børnehave': 'Kindergarten', 'Distriktsskole': 'School'})

# remove duplicates and rows with missing values
filter_df = filter_df.drop_duplicates(subset=['id', 'enhedsnavn', 'type_split'])
filter_df = filter_df[filter_df['longitude'].notna() & filter_df['latitude'].notna()]

# Add noise to the coordinates to avoid overlapping points
filter_df['longitude'] = filter_df['longitude'] + np.random.normal(0, 0.00005, len(filter_df))
filter_df['latitude'] = filter_df['latitude'] + np.random.normal(0, 0.00005, len(filter_df))

# group data to make clickable
grouped = filter_df.groupby('type_EN')

In [178]:
# Create the Folium map for Copenhagen
latitude = 55.676  # Example latitude
longitude = 12.568  # Example longitude
zoom_start = 11.5  # Initial zoom level
m = folium.Map(location=[latitude, longitude], zoom_start=zoom_start, tiles='cartodbpositron')

# plot schools as markers with popups according to the type
f_groups = []

for group_name, group_data in grouped:
    f_groups.append(folium.FeatureGroup(group_name))
    
    for i in range(0,len(group_data)):
    # for index, row in filter_df.iterrows():
        name = group_data['enhedsnavn'].iloc[i]
        coordinates = (group_data['latitude'].iloc[i], group_data['longitude'].iloc[i])
        address = group_data['adresse'].iloc[i]
        link = group_data['www'].iloc[i]
        if pd.isna(link):
            popup_content = '<div style="font-family: Arial, sans-serif; font-size: 10px; color: #333;width: 200px;"><b>' + name + '</b><br>' + group_data['type_EN'].iloc[i] + '<br>' + address + '</div>'
        else:
            popup_content = '<div style="font-family: Arial, sans-serif; font-size: 10px; color: #333;width: 200px;"><b>' + name + '</b><br>' + group_data['type_EN'].iloc[i] + '<br>' + address + '<br><a href="' + link + '" target="_blank">Link</a></div>'

        if 'Nursery' in group_data['type_EN'].iloc[i] or 'Daycare' in group_data['type_EN'].iloc[i]:
            folium.Marker(location=coordinates, popup=popup_content, icon=folium.Icon(color='lightblue', icon='fa-baby', prefix='fa')).add_to(f_groups[-1])
        elif 'Kindergarten' in group_data['type_EN'].iloc[i]:
            folium.Marker(location=coordinates, popup=popup_content, icon=folium.Icon(color='blue', icon='fa-hippo', prefix='fa')).add_to(f_groups[-1])
        elif 'School' in group_data['type_EN'].iloc[i]:
            folium.Marker(location=coordinates, popup=popup_content, icon=folium.Icon(color='darkblue', icon='fa-school', prefix='fa')).add_to(f_groups[-1])

    # Add last featureGroup to Map
    f_groups[-1].add_to(m)

In [179]:
legend_html = '''
     <div style="position: fixed; 
     bottom: 50px; right: 50px; width: 130px; height: 90px; 
     border:2px solid grey; z-index:9999; font-size:12px; font-family: Arial, sans-serif;
     background-color:white;"
     >&nbsp; <i class="fa fa-baby fa-2x" style="color:#83d8ff"></i> &nbsp; Nursery/daycare<br>
     &nbsp; <i class="fa fa-hippo fa-2x" style="color:#36a3d3"></i> &nbsp; Kindergarten<br>
     &nbsp; <i class="fa fa-school fa-2x" style="color:#0065a1"></i> &nbsp; School<br>
      </div>
     ''' 

# Add LayerControl to the map
folium.LayerControl().add_to(m)

# add legend and layer control to the map
m.get_root().html.add_child(folium.Element(legend_html))
m.get_root().html.add_child(folium.Element('<style>.leaflet-control-layers-list, .leaflet-control-layers-overlays {font-size: 12px; font-family: Arial, sans-serif; color: #333;}</style>'))

# m.save('map.html')

In [180]:
# Display the map
m